In [133]:
import numpy as np
import pandas as pd
from math import ceil
from sklearn.cluster import KMeans
import re
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import r2_score
from datetime import datetime

In [134]:
datafile='E:\python大数据\RFM.xlsx'
data=pd.read_excel(datafile)
data.head()

,user_id,gender,last_pay_time,pay_num,pay_times,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,online_time,age
0,Gx4sJzcQog01UhZL,1,2016-06-26 00:00:00,300.04,2,4096.000000,1392.038508,108.100000,0.000000,564.366667,85,31
1,kEXrhTiug93DIcLG,1,2016-06-26 00:00:00,300.00,3,0.000000,62852.509718,240.100000,355.166667,0.000000,10,30
2,AouXr0EOUtSRdiYK,2,2016-06-19 00:00:00,50.00,4,0.000000,1935.242104,27.666667,0.000000,98.416667,12,25
3,Yds7U30hnRZDiLtb,1,2016-06-16 00:00:00,100.00,1,37.336425,988.561075,89.900000,74.483333,121.833333,134,44
4,OFDTSXrhN9Q2mbVw,1,2016-06-27 00:00:00,1000.03,12,3305.741127,0.000000,0.000000,35.100000,496.733333,84,31


In [135]:
data.shape#返回行数列数

(301, 12)

In [136]:
data.describe()#两个核心数据结构的统计变量

,gender,pay_num,pay_times,last_month_traffic,local_trafffic_month,local_caller_time,service1_caller_time,service2_caller_time,online_time,age
count,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000
mean,1.199336,119.214518,1.990033,447.762999,6065.336874,47.652326,33.913234,95.810742,39.813953,31.215947
std,0.496789,148.409316,1.519836,1122.077105,11290.348491,85.436413,81.736976,140.865125,44.629198,12.230149
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000
25%,1.000000,42.000000,1.000000,0.000000,22.655023,0.000000,0.000000,0.000000,10.000000,24.000000
50%,1.000000,80.000000,2.000000,0.000000,1483.326315,7.883333,0.000000,28.566667,17.000000,30.000000
75%,1.000000,150.000000,2.000000,500.000000,5810.480609,58.566667,26.733333,158.033333,59.000000,39.000000
max,2.000000,1477.600000,12.000000,11264.000000,75701.775427,567.900000,675.650000,809.616667,249.000000,70.000000


In [137]:
(data == 0).any()

user_id                 False
gender                   True
last_pay_time           False
pay_num                  True
pay_times                True
last_month_traffic       True
local_trafffic_month     True
local_caller_time        True
service1_caller_time     True
service2_caller_time     True
online_time             False
age                      True
dtype: bool

In [138]:
for col in data.columns:
    #count从0开始累加
    count = 0
    #如果值为0，count累加
    count = [count + 1 for x in data[col] if x == 0]
    #输出该列的0值个数
    print(col+' '+str(sum(count)))

user_id 0
gender 13
last_pay_time 0
pay_num 2
pay_times 6
last_month_traffic 205
local_trafffic_month 68
local_caller_time 92
service1_caller_time 164
service2_caller_time 101
online_time 0
age 12


In [139]:
index3 = ((data["gender"] == 0) |(data["age"] == 0)) | ((data["pay_num"] == 0) | (data["pay_times"] == 0))
#删除13条记录
data = data.drop(data[index3].index)
data.shape

(285, 12)

In [140]:
data.duplicated().sum()

0

In [141]:
data.duplicated(['user_id']).sum()

2

In [142]:
data=data.drop_duplicates(['user_id'])
data.shape

(283, 12)

In [143]:
#筛选属性列
data_select=data[['user_id','pay_num','pay_times','last_pay_time']]
data_select.head()

#重命名列名
data_select.columns = ['用户id','消费金额','消费次数','最后一次消费时间']
data_select.head()

,用户id,消费金额,消费次数,最后一次消费时间
0,Gx4sJzcQog01UhZL,300.04,2,2016-06-26 00:00:00
1,kEXrhTiug93DIcLG,300.00,3,2016-06-26 00:00:00
2,AouXr0EOUtSRdiYK,50.00,4,2016-06-19 00:00:00
3,Yds7U30hnRZDiLtb,100.00,1,2016-06-16 00:00:00
4,OFDTSXrhN9Q2mbVw,1000.03,12,2016-06-27 00:00:00


In [144]:


#生成起始日期和提数日日期
exdata_date = datetime(2016,7,20)
start_date = datetime(2016,6,1)

#输出两个日期
print(exdata_date)
print(start_date)

2016-07-20 00:00:00
2016-06-01 00:00:00


In [145]:
#转化为可计算的日期类型数据
data_select['最后一次消费时间'] = pd.to_datetime(data_select['最后一次消费时间'])

#计算R（最后一次消费距提数日时间）
data_select['R'] = exdata_date - data_select['最后一次消费时间']
data_select['最后一次消费时间']

C:\Users\w\AppData\Local\Temp\ipykernel_16272\2849470151.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select['最后一次消费时间'] = pd.to_datetime(data_select['最后一次消费时间'])
C:\Users\w\AppData\Local\Temp\ipykernel_16272\2849470151.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select['R'] = exdata_date - data_select['最后一次消费时间']


0     2016-06-26
1     2016-06-26
2     2016-06-19
3     2016-06-16
4     2016-06-27
         ...    
296   2016-06-19
297   2016-06-24
298   2016-06-30
299   2016-06-04
300   2016-07-05
Name: 最后一次消费时间, Length: 283, dtype: datetime64[ns]

In [146]:


#计算最后一次消费时间和起始日期的天数差
period_day = data_select['最后一次消费时间'] - start_date
#创建空列表统计月数
period_month = []

#遍历天数，向上取整生成月数
for i in period_day:
    period_month.append(ceil(i.days/30))

period_month

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,


In [147]:
#第一次输出月数统计
print(period_month)

#分割线
print("#"*110)

#遍历清除0值
for i in range(0,len(period_month)):
#如果有月份值为0，令其为1
    if period_month[i] == 0:
        period_month[i]=1

#第二次输出月数统计
print(period_month)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2]
##############################################################################################################
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [148]:
data_select['F(月均消费次数)']=data_select['消费次数']/period_month#月均消费次数
data_select['M(月均消费金额)']=data_select['消费金额']/period_month#月均消费金额
data_select

C:\Users\w\AppData\Local\Temp\ipykernel_16272\1684445261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select['F(月均消费次数)']=data_select['消费次数']/period_month#月均消费次数
C:\Users\w\AppData\Local\Temp\ipykernel_16272\1684445261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select['M(月均消费金额)']=data_select['消费金额']/period_month#月均消费金额


,用户id,消费金额,消费次数,最后一次消费时间,R,F(月均消费次数),M(月均消费金额)
0,Gx4sJzcQog01UhZL,300.04,2,2016-06-26,24 days,2.0,300.04
1,kEXrhTiug93DIcLG,300.00,3,2016-06-26,24 days,3.0,300.00
2,AouXr0EOUtSRdiYK,50.00,4,2016-06-19,31 days,4.0,50.00
3,Yds7U30hnRZDiLtb,100.00,1,2016-06-16,34 days,1.0,100.00
4,OFDTSXrhN9Q2mbVw,1000.03,12,2016-06-27,23 days,12.0,1000.03
...,...,...,...,...,...,...,...
296,qpXPSkahTJ4QnKCO,200.07,3,2016-06-19,31 days,3.0,200.07
297,LPdyxMrDVoa4K5cC,50.00,1,2016-06-24,26 days,1.0,50.00
298,pBUMdi2P8NhTYcj3,30.00,1,2016-06-30,20 days,1.0,30.00
299,5aNKrc6jFdfZvqus,130.00,4,2016-06-04,46 days,4.0,130.00


In [149]:
#标准化处理
user_id=data_select.用户id
data_select=data_select[['R','F(月均消费次数)','M(月均消费金额)']]
data_select.R=data_select.R.map(lambda x : float(x.days))
data_select= (data_select- data_select.mean())/data_select.std()

#重命名列名
data_select.columns = ['R(标准化)','F(标准化)','M(标准化)']
data_select=data_select.join(user_id)
data_select.head()

C:\Users\w\AppData\Local\Temp\ipykernel_16272\1728647155.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_select.R=data_select.R.map(lambda x : float(x.days))


,R(标准化),F(标准化),M(标准化),用户id
0,-0.020365,0.169045,1.452731,Gx4sJzcQog01UhZL
1,-0.020365,0.867433,1.452436,kEXrhTiug93DIcLG
2,0.610008,1.565821,-0.387261,AouXr0EOUtSRdiYK
3,0.880169,-0.529344,-0.019322,Yds7U30hnRZDiLtb
4,-0.110419,7.152927,6.603810,OFDTSXrhN9Q2mbVw


In [150]:
data_select.to_excel('E:\python大数据\class13.xlsx')